In [ ]:
import os
import cv2
import torch
import numpy as np
import torchvision
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# Data Preparation

In [ ]:
training_folder = "/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Training Data"
print("Number of classes are ", len(os.listdir(training_folder)))

count = 0 

for i in os.listdir(training_folder):
    folder_path = os.path.join(training_folder, i)
    print("Number of images in {} are : {}".format(i, len(os.listdir(folder_path))))
    count += len(os.listdir(folder_path))                                            
    
print("Total Training Images are ", count)   

In [ ]:
validation_folder = "/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Validation Data"
print("Number of classes are ", len(os.listdir(training_folder)))

count = 0 

for i in os.listdir(validation_folder):
    folder_path = os.path.join(validation_folder, i)
    print("Number of images in {} are : {}".format(i, len(os.listdir(folder_path))))
    count += len(os.listdir(folder_path))
    
print("Total Validation Images are ", count) 

In [ ]:
x_train = []
y_train = []

for i in tqdm(os.listdir(training_folder)):
    for j in tqdm(os.listdir(os.path.join(training_folder,i))):
        #clss = i
        img_path = os.path.join(os.path.join(training_folder,i), j)
        img = cv2.imread(img_path)
        x_train.append(img)
        if(i=="Coast"):
            clss = 0
        elif(i=="Desert"):
            clss=1
        elif(i=="Forest"):
            clss=2
        elif(i=="Glacier"):
            clss=3
        elif(i=="Mountain"):
            clss=4
        y_train.append(clss)

In [ ]:
x_val = []
y_val = []

for i in tqdm(os.listdir(validation_folder)):
    for j in tqdm(os.listdir(os.path.join(validation_folder,i))):
        #clss = i
        img_path = os.path.join(os.path.join(validation_folder,i), j)
        img = cv2.imread(img_path)
        x_val.append(img)
        if(i=="Coast"):
            clss = 0
        elif(i=="Desert"):
            clss=1
        elif(i=="Forest"):
            clss=2
        elif(i=="Glacier"):
            clss=3
        elif(i=="Mountain"):
            clss=4
        y_val.append(clss)

# MobileNet V3 Small

In [ ]:
model = torchvision.models.mobilenet_v3_small(pretrained=True)

In [ ]:
for layers in model.parameters():
    layers.requires_grad=False
new_classifier = torch.nn.Sequential(torch.nn.Linear(576,1024),
                                     torch.nn.Hardswish(),
                                     torch.nn.Dropout(p=0.2, inplace=True),
                                     torch.nn.Linear(1024, 5))
model.classifier = new_classifier

device = torch.device("cuda")
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)

In [ ]:
class data(Dataset):
    def __init__(self, data, labels, transform=None):
        self.dataset = data
        self.transform = transform
        self.label = labels
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self,index):
        image = self.dataset[index]
        #image = image.reshape(100,100)
        #image = cv2.resize(image, (224,224))
        #image = image/255
        #image = image.astype(np.float32)
        target = self.label[index]
        if(self.transform is not None):
            image = self.transform(image)
        return image, target


train_transform = torchvision.transforms.Compose(
                                            [
                                            torchvision.transforms.ToPILImage(),
                                            torchvision.transforms.RandomHorizontalFlip(),
                                            torchvision.transforms.RandomResizedCrop(224),
                                            #torchvision.transforms.RandomGrayscale(),
                                            #torchvision.transforms.GaussianBlur(kernel_size=2),
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(mean=0.5, std=0.5)
                                           ]
                                                )
                                           #torchvision.transforms.Grayscale()])

val_transform = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(),
                                                torchvision.transforms.Resize((224,224)),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize(mean=0.5, std=0.5)
                                              ])    
    
    
train_data = data(x_train, y_train, train_transform)
val_data = data(x_val, y_val, val_transform)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=True)

In [ ]:
lrs = []
val_acc = []
train_acc = []
ep_loss = []
for epoch in range(0,10):
    print("Epoch : ",epoch)
    epoch_loss = 0
    batches = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        batches+=1
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs=model(inputs)
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    lrs.append(optimizer.param_groups[0]["lr"])
    scheduler.step()
    inputs,targets = iter(val_loader).next()
    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = model(inputs).detach().cpu().numpy().argmax(axis=-1)
    val_accuracy = (outputs == targets.cpu().numpy()).mean()
    val_acc.append(val_accuracy)
    train_accuracy = 0
    for inputs,targets in tqdm(train_loader):
        inputs = inputs.to(device)
        outputs = model(inputs).detach().cpu().numpy().argmax(axis=-1)
        train_accuracy += (outputs == targets.numpy()).mean()
    train_accuracy = train_accuracy/batches
    train_acc.append(train_accuracy)
    #ep_loss.append(epoch_loss.detach().cpu().numpy())
    ep_loss.append(epoch_loss)
    print("Train accuracy on Batch of {} images is {} ".format(batches, train_accuracy))
    print("Validation accuracy on Batch of {} images is {}".format(batches, val_accuracy))
    print("Loss is ", epoch_loss)

In [ ]:
plt.style.use("ggplot")
plt.plot(train_acc, label="Training accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend()
plt.show()

# Evaluating with the testing data

In [ ]:
infer_model = model.eval()

In [ ]:
correct, wrong = 0, 0
testing_folder = "/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Testing Data"
for i in os.listdir(testing_folder):
    if(i=="Coast"):
        actual_clss = 0
    elif(i=="Desert"):
        actual_clss=1
    elif(i=="Forest"):
        actual_clss=2
    elif(i=="Glacier"):
        actual_clss=3
    elif(i=="Mountain"):
        actual_clss=4
    folder_path = os.path.join(testing_folder, i)
    for j in tqdm(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, j)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,224))
        img = img.astype(np.float32)
        img = img/255
        img = torch.Tensor(img)
        img = img.permute(2,0,1).unsqueeze(0)
        img = img.cuda()
        clss = infer_model(img).argmax(axis=-1).item()
        if (actual_clss == clss):
            correct += 1
        else:
            wrong +=1

In [ ]:
#Total Accuracy
correct/(correct+wrong)